## Diagnostic Notebook for `enable_git_proxy_jupyter_v0.1.0.ipynb`

### 1. Checking the Running Logs

In [ ]:
!cat /databricks/git-proxy/git-proxy.log

### 2. Internet Connection Test

In [ ]:

import requests

def can_access_internet():
    try:
        response = requests.get("https://www.google.com")
        response.raise_for_status()
        print("Internet access confirmed.")
    except:
        print("Cannot access the internet. Check your connection.")

can_access_internet()


### 3. Check SSL Certificate Configuration

In [ ]:

import os

certificate_path = os.environ.get('GIT_PROXY_CA_CERT_PATH')
if certificate_path:
    print(f"Certificate path is set to: {certificate_path}")
    if os.path.exists(certificate_path):
        print("Certificate file exists.")
    else:
        print("Certificate file doesn't exist. Please check the path.")
else:
    print("GIT_PROXY_CA_CERT_PATH is not set.")


### 4. Test Connectivity to Internal Git Server

In [ ]:

internal_git_server = input("GIT_PROXY_TEST_URL:")
if internal_git_server:
    try:
        response = requests.get(internal_git_server)
        response.raise_for_status()
        print(f"Successfully connected to the internal Git server at {internal_git_server}.")
    except requests.RequestException as e:
        print(f"Failed to connect to the internal Git server at {internal_git_server}. Reason: {e}")
else:
    print("Internal Git server URL (GIT_PROXY_TEST_URL) is not set.")


### 5. Check SSL Verification Configuration

In [ ]:

enable_ssl_verification = os.environ.get("GIT_PROXY_ENABLE_SSL_VERIFICATION", "True") == "True"
if enable_ssl_verification:
    print("SSL verification is enabled.")
else:
    print("SSL verification is disabled. If facing issues, consider enabling it.")


##6. Check custom SSL configuration

In [ ]:
import os
import requests
import certifi

def print_and_exit(message):
    """Print a message and exit."""
    print(message)
    exit(0)

def can_access_internet():
    """Check if we can access the internet."""
    try:
        requests.get("https://www.google.com")
        return True
    except:
        return False

def test_connection(url, cert_path=None):
    """Test if we can connect to a URL, optionally using a certificate."""
    try:
        requests.get(url, verify=cert_path if cert_path else True)
        return True
    except requests.RequestException:
        return False

# Check internet access
if not can_access_internet():
    print_and_exit("[ERROR] Can't access the internet. Check your connection.")

# Fetch environment settings
certificate_path = os.environ.get('GIT_PROXY_CA_CERT_PATH')
enable_ssl_verification = os.environ.get("GIT_PROXY_ENABLE_SSL_VERIFICATION", "True") == "True"
internal_test_url = os.environ.get("GIT_PROXY_TEST_URL")

# If SSL verification is disabled, we're done
if not enable_ssl_verification:
    print_and_exit("[NOTE] SSL verification is disabled. If facing issues, consider enabling it.")

# Ensure a certificate path is provided
if not certificate_path:
    print_and_exit("[ERROR] GIT_PROXY_CA_CERT_PATH not set. Please set it.")
else:
    # Display the certificate for reference
    with open(certificate_path, 'r') as file:
        print("\n--- CERTIFICATE CONTENT ---")
        print(file.read())
        print("---------------------------\n")

    # Test connectivity to a known URL using the provided certificate
    github_url = "https://github.com/databricks/databricks-repos-proxy.git"
    if test_connection(github_url, certificate_path):
        print(f"[SUCCESS] Connected to {github_url} using the provided certificate.")
    else:
        # Combine system certificates with provided certificate
        combined_cert_path = "/tmp/combined_cert.pem"
        with open(combined_cert_path, 'w') as combined_file:
            with open(certifi.where(), 'r') as system_certs:
                combined_file.write(system_certs.read())
            with open(certificate_path, 'r') as custom_cert:
                combined_file.write(custom_cert.read())

        if test_connection(github_url, combined_cert_path):
            print(f"[SUCCESS] Connected to {github_url} using combined system and custom certificates.")
        else:
            print(f"[ERROR] Failed to connect to {github_url} even with combined certificates. Check certificate validity.")

    # If an internal test URL is provided, test it as well
    if internal_test_url:
        if test_connection(internal_test_url, certificate_path):
            print(f"[SUCCESS] Connected to internal URL {internal_test_url} using the provided certificate.")
        else:
            if test_connection(internal_test_url, combined_cert_path):
                print(f"[SUCCESS] Connected to internal URL {internal_test_url} using COMBINED certificates.")
            else:
                print(f"[ERROR] Failed to connect to the internal URL {internal_test_url} even with combined certificates.")